<a href="https://colab.research.google.com/github/halfag/LocalLLM/blob/main/Mixtral_Autogen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!apt-get update -q && apt install pciutils -y

!curl https://ollama.ai/install.sh | sh
!nohup ollama serve &




In [2]:
%%capture
!ollama pull dolphin-mixtral

In [3]:
%%capture
!pip install kaleido python-multipart cohere
!pip install litellm[proxy_server] pyautogen


In [4]:
%%capture
!nohup litellm --model ollama/dolphin-mixtral &

In [ ]:
!pip install aiohttp pyngrok
!ngrok config add-authtoken 2YyeNPLWs82ZVtL13BIV3rZldWj_68suxzPW6XuS7F1fGfeko

import os
import asyncio

# Set LD_LIBRARY_PATH so the system NVIDIA library
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

async def run_process(cmd):
  print('>>> starting', *cmd)
  p = await asyncio.subprocess.create_subprocess_exec(
      *cmd,
      stdout=asyncio.subprocess.PIPE,
      stderr=asyncio.subprocess.PIPE,
  )

  async def pipe(lines):
    async for line in lines:
      print(line.strip().decode('utf-8'))

  await asyncio.gather(
      pipe(p.stdout),
      pipe(p.stderr),
  )

await asyncio.gather(
    run_process(['ollama', 'serve']),
    run_process(['ngrok', 'http', '--log', 'stderr', '11434']),
)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
>>> starting ollama serve
>>> starting ngrok http --log stderr 11434
Error: listen tcp 127.0.0.1:11434: bind: address already in use
t=2023-12-21T16:42:40+0000 lvl=info msg="no configuration paths supplied"
t=2023-12-21T16:42:40+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml
t=2023-12-21T16:42:40+0000 lvl=info msg="open config file" path=/root/.config/ngrok/ngrok.yml err=nil
t=2023-12-21T16:42:40+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040 allow_hosts=[]
t=2023-12-21T16:42:41+0000 lvl=info msg="client session established" obj=tunnels.session obj=csess id=538f0b1481cb
t=2023-12-21T16:42:41+0000 lvl=info msg="tunnel session started" obj=tunnels.session
t=2023-12-21T16:42:41+0000 lvl=info msg="started tunnel" obj=tunnels name=command_line addr=http://localhost:11434 url=https://9bb0-34-142-205-14.ngrok-free.app


In [ ]:
# Code used:
import autogen
import openai
openai.api_type = "open_ai"
openai.api_key = "sk-111111111111111111111111111111111111111111111"
openai.api_base = "http://127.0.0.1:8000/v1"
# openai.api_version = "2023-05-15"


wizard_config_list = [
    {
        "api_key": "sk-111111111111111111111111111111111111111111111",
        #"api_key": "NULL",
        # 'api_type': 'open_ai',
        'base_url': "http://127.0.0.1:8000/v1",
        # 'api_version': '2023-05-15',
        # "timeout": "600",
        # "request_timeout": 8000
    },
]

llama2_config_list = [
    {
        "base_url": "http://127.0.0.1:8000/v1",
        #"api_key": "NULL",
        "api_key": "sk-111111111111111111111111111111111111111111111",
        # 'api_type': 'open_ai',
        # "timeout": "600",
        # "request_timeout": 8000
    },
]


coder_llm_config = {
    # "seed": None,  # change the seed for different trials
    "temperature": 0,
    "config_list": llama2_config_list,
    # "timeout": 600,
    # "request_timeout": 8000
    # "clear_cache": True
}

llama2_llm_config = {
    # "seed": None,  # change the seed for different trials
    # no caching
    # "clear_cache": True,
    "temperature": 0,
    "config_list": llama2_config_list,
    # "timeout": 600,
    # "request_timeout": 8000
}


user_proxy = autogen.UserProxyAgent(
   name="User_proxy",
   system_message = """Reply TERMINATE if the task has been solved at full satisfaction. Otherwise, reply CONTINUE, or the reason why the task is not solved yet.""",
   code_execution_config={"last_n_messages": 2, "work_dir": "groupchat", "use_docker":"True"},
   human_input_mode="TERMINATE"
)
coder = autogen.AssistantAgent(
    name="Coder",
    llm_config=llama2_llm_config
)
pm = autogen.AssistantAgent(
    name="Product_manager",
    system_message="Manage deveolpment of software",
    llm_config=llama2_llm_config
)
groupchat = autogen.GroupChat(agents=[user_proxy, coder, pm], messages=[], max_round=12)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llama2_llm_config)



In [ ]:
user_proxy.initiate_chat(manager, message="""What's today's date?""")
# type exit to terminate the chat